# Treinamento de um Modelo para Classificação de Tweets com Discursos de Ódio

### Contém o código que realiza o fine tuning do BERTimbau e verifica sua acurácia

Membros: Lucas de Medeiros Soares, João Henrique Almeida Xavier e Tarso Jabbes Lima de Oliveira

In [ ]:
import pandas as pd
import os
import zipfile

## Criando o dataframe

In [2]:
df = pd.read_csv('/content/train_data2.csv')

## Pre-processamento do dataframe
- Remoção de espaços em branco seguidos
- Colocar todos os caracteres em minúsculo
- Tranformar as labels em valores inteiros

In [3]:
def preprocess(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df['query'] = df['query'].str.replace('[^\w\s]','')
    df['query'] = df['query'].str.lower()
    df["label"] = df["label"].astype(int)
    return df

df = preprocess(df)

In [ ]:
!pip install datasets

In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)

In [6]:
dataset['train'][1]

{'query': "'você tá gorda demais, tá feia' 'eu te amo, mas você precisa se arrumar mais'",
 'label': 1,
 '__index_level_0__': 1420}

## Criação das funções de tokenização

In [ ]:
# Importa a classe AutoTokenizer da biblioteca transformers, que permite carregar um tokenizer pré-treinado
from transformers import AutoTokenizer

# Carrega o tokenizer pré-treinado para o modelo BERTimbau (BERT em português) da Neuralmind
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Define uma função para tokenizar os dados
def tokenize_function(examples):
    return tokenizer(
        examples["query"],                # A coluna que contém os textos a serem tokenizados
        padding="max_length",             # Preenche com padding até o comprimento máximo especificado
        truncation=True,                   # Corta as sequências que excedem o comprimento máximo
        add_special_tokens=True,          # Adiciona tokens especiais necessários para o modelo (como [CLS] e [SEP])
        max_length=512                     # Define o comprimento máximo das sequências para 512 tokens
    )

# Aplica a função de tokenização ao conjunto de dados, usando batched=True para processar várias entradas ao mesmo tempo
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
# Importa a classe AutoModelForSequenceClassification da biblioteca transformers,
# que permite carregar um modelo pré-treinado para classificação de sequências
from transformers import AutoModelForSequenceClassification, TrainingArguments

# Carrega o modelo BERTimbau pré-treinado para classificação de sequências.
# O parâmetro num_labels=2 indica que o modelo será usado para um problema de classificação binária (2 classes)
model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)


## Definição dos parâmetros para o modelo

In [ ]:
# Instala a biblioteca 'evaluate' para facilitar o cálculo de métricas de avaliação
!pip install evaluate

# Importa a biblioteca 'evaluate' e a função 'numpy' para manipulação de arrays
import evaluate
import numpy as np

# Importa as classes TrainingArguments e Trainer da biblioteca transformers
from transformers import TrainingArguments, Trainer

# Define os argumentos de treinamento para o modelo
training_args = TrainingArguments(
    output_dir="./results",          # Diretório onde os resultados do treinamento serão salvos
    num_train_epochs=5,              # Número de épocas de treinamento
    eval_strategy="epoch",           # Avaliação do modelo após cada época
    save_strategy="no"               # Não salvar o modelo durante o treinamento
)

# Carrega a métrica de acurácia usando a biblioteca 'evaluate'
metric = evaluate.load("accuracy")

# Define uma função para calcular as métricas de avaliação
def compute_metrics(eval_pred):
    predictions, labels = eval_pred  # Desempacota as previsões e os rótulos
    predictions = np.argmax(predictions, axis=1)  # Obtém os índices das classes previstas
    return metric.compute(predictions=predictions, references=labels)  # Calcula e retorna a acurácia

# Cria um objeto Trainer, que gerencia o treinamento e a avaliação do modelo
trainer = Trainer(
    model,                               # O modelo a ser treinado
    training_args,                       # Os argumentos de treinamento definidos acima
    train_dataset=tokenized_datasets["train"],  # Conjunto de dados para treinamento
    eval_dataset=tokenized_datasets["test"],    # Conjunto de dados para avaliação
    tokenizer=tokenizer,                 # Tokenizer usado para o modelo
    compute_metrics=compute_metrics       # Função para calcular as métricas durante a avaliação
)


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.577191,0.718000
2,0.494700,0.629960,0.724000
3,0.494700,1.159619,0.722000
4,0.164100,1.245447,0.716000
5,0.164100,1.455266,0.712000


TrainOutput(global_step=1250, training_loss=0.2785097351074219, metrics={'train_runtime': 1002.9594, 'train_samples_per_second': 9.961, 'train_steps_per_second': 1.246, 'total_flos': 2628479443046400.0, 'train_loss': 0.2785097351074219, 'epoch': 5.0})

## Avaliação do modelo após o treinamento

In [13]:
trainer.evaluate()

{'eval_loss': 1.455265760421753,
 'eval_accuracy': 0.712,
 'eval_runtime': 14.6547,
 'eval_samples_per_second': 34.119,
 'eval_steps_per_second': 4.299,
 'epoch': 5.0}

In [ ]:
trainer.save_model("../model")